In [1]:
!nvidia-smi

Thu Jul 28 09:31:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   47C    P5    16W /  N/A |    668MiB /  6144MiB |     27%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [43]:
!pip show torchtext

Name: torchtext
Version: 0.13.0
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /home/yassine/anaconda3/envs/data-science-stack-2.9.0/lib/python3.7/site-packages
Requires: tqdm, torch, requests, numpy
Required-by: 


In [44]:
!pip show torch

Name: torch
Version: 1.12.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/yassine/anaconda3/envs/data-science-stack-2.9.0/lib/python3.7/site-packages
Requires: typing-extensions
Required-by: torchvision, torchtext, camel-tools, accelerate


In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('val_translation_madar.csv')

In [5]:
import math
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
from torch import Tensor
import io
import time

torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

In [42]:
#change file paths for training validation and test corporas

train_filepaths = ['corp.mor', 'corp.alg']
val_filepaths = ['moroccan_madar_fes_cleaned.csv', 'algerian_madar_cleaned.csv']
test_filepaths = val_filepaths


In [7]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('UBC-NLP/ARBERT', pad_token='[PAD]', eos_token='[EOS]', bos_token='[BOS]',)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='UBC-NLP/ARBERT', vocab_size=100000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[BOS]', 'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [9]:
from torchtext.vocab import vocab

In [10]:
tokenizer.unk_token

'[UNK]'

In [11]:
def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      counter.update(tokenizer(string_))
  return vocab(counter, specials=['[UNK]', '[PAD]', '[BOS]', '[EOS]'])

mor_vocab = build_vocab(train_filepaths[0], tokenizer.tokenize)
alg_vocab = build_vocab(train_filepaths[1], tokenizer.tokenize)



In [12]:
mor_vocab['[PAD]']

1

In [13]:
mor_vocab.set_default_index(mor_vocab['[UNK]'])

In [14]:
alg_vocab.set_default_index(alg_vocab['[UNK]'])

In [15]:
alg_vocab['[EOS]']

3

In [16]:
raw_mor_iter = iter(io.open(train_filepaths[0], encoding="utf8"))
raw_alg_iter = iter(io.open(train_filepaths[1], encoding="utf8"))
data = []
i=0
for (raw_de, raw_en) in zip(raw_mor_iter, raw_alg_iter):
  i=i+1
  if i==1:

    print(raw_de)

ناس بداو تيتباكاو وبديت تنهدر ب نفعل بحال إلى تنهدي الناس



In [17]:
a=tokenizer.encode("ناس بداو تيتباكاو وبديت تنهدر ب نفعل بحال إلى تنهدي الناس")

In [18]:
tokenizer.decode(a)

'[CLS] ناس بداو تيتباكاو وبديت تنهدر ب نفعل بحال الى تنهدي الناس [SEP]'

In [19]:
def data_process(filepaths):
  raw_mor_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_alg_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_mor_iter, raw_alg_iter):
    mor_tensor_ = torch.tensor([mor_vocab[token] for token in tokenizer.tokenize(raw_de)],
                            dtype=torch.long)
    alg_tensor_ = torch.tensor([alg_vocab[token] for token in tokenizer.tokenize(raw_en)],
                            dtype=torch.long)
    data.append((mor_tensor_, alg_tensor_))
  return data


train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


BATCH_SIZE = 128
PAD_IDX = mor_vocab['[PAD]']
BOS_IDX = mor_vocab['[BOS]']
EOS_IDX = mor_vocab['[EOS]']

In [20]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  mor_batch, alg_batch = [], []
  for (mor_item, alg_item) in data_batch:
    mor_batch.append(torch.cat([torch.tensor([BOS_IDX]), mor_item, torch.tensor([EOS_IDX])], dim=0))
    alg_batch.append(torch.cat([torch.tensor([BOS_IDX]), alg_item, torch.tensor([EOS_IDX])], dim=0))
  mor_batch = pad_sequence(mor_batch, padding_value=PAD_IDX)
  alg_batch = pad_sequence(alg_batch, padding_value=PAD_IDX)
  return mor_batch, alg_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [21]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size, bias=False)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [22]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, tokalg_embedding: Tensor):
        return self.dropout(tokalg_embedding +
                            self.pos_embedding[:tokalg_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [23]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [24]:
SRC_VOCAB_SIZE = len(mor_vocab)
TGT_VOCAB_SIZE = len(alg_vocab)
EMB_SIZE = 210

FFN_HID_DIM = 210
BATCH_SIZE = 16

NUM_EPOCHS = 70

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)



In [25]:
def train_epoch(model, train_iter, optimizer):
  model.train()
  losses = 0
  for idx, (src, tgt) in enumerate(train_iter):
      src = src.to(device)
      tgt = tgt.to(device)

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)

      optimizer.zero_grad()

      tgt_out = tgt[1:,:]
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
      loss.backward()

      optimizer.step()
      losses += loss.item()
  return losses / len(train_iter)


def evaluate(model, val_iter):
  model.eval()
  losses = 0
  for idx, (src, tgt) in (enumerate(valid_iter)):
    src = src.to(device)
    tgt = tgt.to(device)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,
                              src_padding_mask, tgt_padding_mask, src_padding_mask)
    tgt_out = tgt[1:,:]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    losses += loss.item()
  return losses / len(val_iter)

In [40]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys


def translate(model, src, src_vocab, tgt_vocab, tokenizer):
  model.eval()
  tokens = [BOS_IDX] + [src_vocab[tok] for tok in tokenizer.tokenize(src)]+ [EOS_IDX]
  num_tokens = len(tokens)
  src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
  tgt_tokens=[tgt_vocab.get_itos()[tok] for tok in tgt_tokens]
  
  return tokenizer.convert_tokens_to_string(tgt_tokens).replace("[BOS]", "").replace("[EOS]", "").strip()

In [27]:
import sacrebleu

In [28]:
from statistics import mean

In [38]:
list_head=[3,5,6,7]
list_layers=[1,2,3]

In [39]:
for head in list_head:
  sourcefile=open('output', 'w')
  NHEAD=head
  for layers in list_layers:
    NUM_ENCODER_LAYERS=layers
    NUM_DECODER_LAYERS=layers
    transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

    for p in transformer.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)

    transformer = transformer.to(device)
    optimizer = torch.optim.Adam(
      transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
      )
    for epoch in range(1, NUM_EPOCHS+1):
      start_time = time.time()
      train_loss = train_epoch(transformer, train_iter, optimizer)
      val_loss = evaluate(transformer, valid_iter)
      end_time = time.time()
      print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
      
    preds=[translate(transformer, sentence, mor_vocab, alg_vocab, tokenizer) for sentence in df.mor.values]
    labels=df.alg.values
    preds=np.array(preds, dtype=object)
    result=mean([sacrebleu.sentence_bleu(pred, [label]).score for pred, label in zip(preds, labels)])
    print(f"Number of heads: {NHEAD}, Number of layers: {layers}, result: {result}\n", file=sourcefile)
  
  sourcefile.close()

KeyboardInterrupt: 